# dataset

In [30]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("yelp_review_full")
print(type(dataset))
print(dataset.keys())

sub_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
sub_test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))

<class 'datasets.dataset_dict.DatasetDict'>
dict_keys(['train', 'test'])


In [ ]:
sub_train_dataset[100]

{'label': 3,
 'text': "I recently brough my car up to Edinburgh from home, where it had sat on the drive pretty much since I had left home to go to university.\\n\\nAs I'm sure you can imagine, it was pretty filthy, so I pulled up here expecting to shell out \\u00a35 or so for a crappy was that wouldnt really be that great.\\n\\nNeedless to say, when I realised that the cheapest was was \\u00a32, i was suprised and I was even more suprised when the car came out looking like a million dollars.\\n\\nVery impressive for \\u00a32, but thier prices can go up to around \\u00a36 - which I'm sure must involve so many polishes and waxes and cleans that dirt must be simply repelled from the body of your car, never getting dirty again."}

# tokenizer

In [34]:
from transformers import AutoTokenizer

In [35]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [55]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map: 100%|██████████| 50000/50000 [00:07<00:00, 6469.37 examples/s]


In [56]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


In [57]:
len(set(small_train_dataset["label"]))

5

# model

In [75]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased", num_labels=5, torch_dtype="auto"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# pipeline

In [76]:
from transformers import pipeline


In [77]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


Device set to use mps:0


In [78]:
result = classifier("This restaurant has excellent service and great food!")
print(result)


[{'label': 'LABEL_0', 'score': 0.31116628646850586}]


# train

## arguments

---

### **`TrainingArguments` 参数详解（通俗版）**

`TrainingArguments` 是 Hugging Face Transformers 库中控制模型训练的核心配置，直接决定训练速度、显存占用、模型效果等。以下分模块介绍所有重要参数及其影响，用“🚀”标注常见配置场景。

---

### **一、基础训练配置**
#### 1. **`output_dir`**
- **作用**：模型和训练日志的保存路径。
- **示例**：`output_dir="./my_model"` → 训练后的模型会存在 `my_model` 文件夹。
- **注意**：路径要提前创建好，否则可能报错。

#### 2. **`num_train_epochs`**
- **作用**：训练的总轮数（所有数据过一遍算一轮）。
- **影响**：
  - 值太小 → 模型没学完，效果差。
  - 值太大 → 过拟合（训练集表现好，测试集差）。
- **🚀推荐**：微调任务通常 3-5 轮，预训练任务 10+ 轮。

#### 3. **`per_device_train_batch_size`**
- **作用**：每个 GPU 一次处理的样本数。
- **影响**：
  - 值越大 → 显存占用越高，训练速度越快。
  - 值太小 → 显存不足时会报错。
- **🚀推荐**：根据显存调整（24GB GPU 可设 4-8）。

#### 4. **`gradient_accumulation_steps`**
- **作用**：攒多少步的梯度再更新模型（等效扩大批量大小）。
- **示例**：`batch_size=2` + `gradient_accumulation=4` → 实际批量=8。
- **🚀场景**：显存不足时用此参数“模拟”大批量训练。

#### 5. **`fp16` / `bf16`**
- **作用**：开启半精度训练（16 位浮点数），节省显存并加速。
  - `fp16`：适合 NVIDIA GPU。
  - `bf16`：适合 AMD GPU 或 TPU。
- **注意**：可能导致数值不稳定（部分模型需关闭）。

---

### **二、优化器与学习率**
#### 6. **`learning_rate`**
- **作用**：学习步长，决定参数更新幅度。
- **影响**：
  - 太大 → 训练震荡甚至崩溃。
  - 太小 → 收敛慢，效果差。
- **🚀推荐**：预训练用 `1e-4`，微调用 `2e-5` ~ `5e-5`。

#### 7. **`weight_decay`**
- **作用**：权重衰减（L2 正则化），防止过拟合。
- **影响**：值越大 → 模型参数越小，泛化性越强，但可能欠拟合。
- **🚀推荐**：默认 `0.01`，复杂任务可调低至 `0.001`。

#### 8. **`optim`**
- **作用**：选择优化器类型。
- **可选值**：
  - `"adamw_torch"`（默认）：适合大多数场景。
  - `"adafactor"`：省显存但收敛慢。
  - `"sgd"`：简单任务可用。
- **🚀推荐**：无特殊需求保持默认。

---

### **三、评估与保存**
#### 9. **`evaluation_strategy`**
- **作用**：何时进行评估。
- **可选值**：
  - `"no"`：不评估。
  - `"steps"`：按步数评估。
  - `"epoch"`：每轮结束时评估。
- **🚀推荐**：验证集较大时用 `"epoch"`，否则用 `"steps"`。

#### 10. **`eval_steps`**
- **作用**：每训练多少步评估一次（需 `evaluation_strategy="steps"`）。
- **示例**：`eval_steps=500` → 每 500 步验证一次。
- **🚀推荐**：根据总步数调整（如总步 5000 → 设 500）。

#### 11. **`save_strategy`**
- **作用**：何时保存模型。
- **可选值**：同 `evaluation_strategy`。
- **🚀技巧**：设为与 `evaluation_strategy` 相同，方便保存最佳模型。

#### 12. **`save_total_limit`**
- **作用**：最多保存多少个模型检查点。
- **示例**：`save_total_limit=3` → 只保留最新的 3 个模型，自动删除旧的。
- **🚀推荐**：设为 2-3，避免占满磁盘。

---

### **四、学习率调度与预热**
#### 13. **`lr_scheduler_type`**
- **作用**：学习率调整策略。
- **可选值**：
  - `"linear"`（默认）：线性衰减。
  - `"cosine"`：余弦退火，适合长时间训练。
  - `"constant"`：固定学习率。
- **🚀推荐**：微调用 `"linear"`，预训练用 `"cosine"`。

#### 14. **`warmup_steps`** / `warmup_ratio`
- **作用**：学习率预热步数（前期逐步增加学习率）。
- **影响**：避免初期大学习率破坏预训练权重。
- **🚀推荐**：设 `warmup_ratio=0.1`（预热 10% 的总步数）。

---

### **五、日志与调试**
#### 15. **`logging_dir`** / `logging_steps`
- **作用**：日志保存路径和记录频率。
- **示例**：`logging_steps=10` → 每 10 步记录一次损失。
- **🚀技巧**：用 TensorBoard 查看日志：`tensorboard --logdir=./logs`。

#### 16. **`report_to`**
- **作用**：日志上报平台。
- **可选值**：`"tensorboard"`（默认）、`"wandb"`（Weights & Biases）。
- **🚀推荐**：本地调试用 `"tensorboard"`，团队协作用 `"wandb"`。

#### 17. **`dataloader_num_workers`**
- **作用**：数据加载的并行进程数。
- **影响**：值越大 → 数据加载越快，但占用更多 CPU 内存。
- **🚀推荐**：设为 CPU 核心数的 70%（如 8 核 → 设 6）。

---

### **六、恢复训练与设备管理**
#### 18. **`resume_from_checkpoint`**
- **作用**：从指定检查点恢复训练。
- **示例**：`resume_from_checkpoint="./my_model/checkpoint-500"`。
- **🚀场景**：训练意外中断后继续训练。

#### 19. **`gradient_checkpointing`**
- **作用**：启用梯度检查点，用时间换显存。
- **影响**：显存减少 60%，但训练速度下降 20%。
- **🚀推荐**：显存不足时开启（如训练 10B+ 模型）。

#### 20. **`device_map`**
- **作用**：指定模型加载到哪些设备（GPU/CPU）。
- **示例**：`device_map="auto"` → 自动分配模型层到可用 GPU。
- **🚀注意**：需安装 `accelerate` 库。

---

### **完整总结表格**
| **参数** | **常见值** | **核心影响** | **适用场景** |
|----------|------------|--------------|--------------|
| `per_device_train_batch_size` | 2-8 | 显存占用 vs 速度 | 根据 GPU 显存调整 |
| `gradient_accumulation_steps` | 4-8 | 模拟大批量训练 | 显存不足时扩大批量 |
| `learning_rate` | 2e-5 ~ 5e-5 | 收敛速度 vs 稳定性 | 微调任务常用 |
| `fp16` | `True` | 省显存 + 加速 | NVIDIA GPU 必开 |
| `warmup_steps` | 总步数的 10% | 避免初期震荡 | 所有训练任务 |
| `save_total_limit` | 2-3 | 控制磁盘占用 | 长期训练任务 |

---

### **配置示例**
```python
# 单卡 24GB GPU 微调配置（如 LLaMA-7B）
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 实际批量=16
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200,
    warmup_steps=50,
    weight_decay=0.01,
    optim="adamw_torch",
    report_to="tensorboard"
)
```

In [60]:
from transformers import TrainingArguments

In [61]:
training_args = TrainingArguments(output_dir="test_trainer")

## evaluate

In [62]:
import numpy as np
import evaluate



In [63]:
metric = evaluate.load("accuracy")

In [64]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [72]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  eval_strategy="epoch",
                                  num_train_epochs=0.01,
                                  )

## trainer

In [73]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)


In [74]:
trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("test_trainer")
tokenizer.save_pretrained("test_trainer")